In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import bs4
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from time import sleep
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException
import pandas as pd 
import re

In [5]:
url = "https://www.zapimoveis.com.br/aluguel/apartamentos/"

chrome_option = webdriver.ChromeOptions()
chrome_option.add_argument('--headless')

driver = webdriver.Chrome('chromedriver')
driver.get(url)
soup = BeautifulSoup(driver.page_source, "html.parser")


driver.find_element(By.ID, "cookie-notifier-cta").click()
dados = []

while True:
    try:
        card = soup.find_all('div', class_='card-listing simple-card')
        for resgistro in card:

            valor = "".join([c for c in resgistro.find('div', 'oz-datazap-stamp').text if c.isdigit()])
            descricao = resgistro.find('span', class_='simple-card__text text-regular')
            descricao = descricao.text if descricao else "null"

            if resgistro.find('span', 'card-price__value') == None: condominio = "Null"
            else: condominio = resgistro.find('span', 'card-price__value').text
            
            if resgistro.find('li', 'card-price__item iptu text-regular') == None: iptu = "Null"
            else: iptu = "".join([c for c in resgistro.find('li', 'card-price__item iptu text-regular').text if c.isdigit()])
            
            local = resgistro.find('h2', 'simple-card__address color-dark text-regular').text.strip()
            tamamho = "".join([c for c in resgistro.find('li', 'feature__item text-small js-areas').text if c.isdigit()])
            quartos = "".join([c for c in resgistro.find('li', 'feature__item text-small js-bedrooms').text if c.isdigit()])
                
            if resgistro.find('li', 'feature__item text-small js-parking-spaces') == None: vagas_garagem = "Null"
            else: vagas_garagem = "".join([c for c in resgistro.find('li', 'feature__item text-small js-parking-spaces').text if c.isdigit()])
            banheiros = "".join([c for c in resgistro.find('li', 'feature__item text-small js-bathrooms').text if c.isdigit()])
            novidade = resgistro.find('div', class_='simple-card__highligths')
            novidade = novidade.text if novidade else 'null'

            dados.append([valor, descricao, condominio, iptu, local, tamamho, quartos, vagas_garagem, banheiros, novidade])

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(5)
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="Próxima Página"]'))
        )
        print('indo para proxima pagina')
        next_button.click()

    except Exception as e:
        print('Fim da raspagem')
        driver.quit()
        break
        

In [6]:
df = pd.DataFrame(data=dados, columns=['valor', 'descricao', 'condominio', 'iptu', 'local', 'tamamho', 'quartos', 'vagas_garagem', 'banheiros', 'novidade'])
display(df)

,valor,descricao,condominio,iptu,local,tamamho,quartos,vagas_garagem,banheiros,novidade
0,2000,"Apartamento de 54m², 1 dormitório com armários...",R$ 650,70,"Rua Carlos Escobar, Santana",54²,1,1,1,Super destaque
1,3200,Alugue sem fiador ou caução! Contrato digital!...,R$ 300,Null,"Rua Antonio Perez Hernandez, Parque Campolim",80²,3,2,2,Super destaque
2,1800,Apartamento semimobiliado com excelente locali...,R$ 350,31,"Rua Euclydes Miranda, Partenon",64²,2,1,1,Super destaque
3,2400,APTO COM LOCALIZAÇÃO ESPETACULAR PARA LOCAÇÃO!...,R$ 522,40,"Rua Riachuelo, Vila Assuncao",60²,2,2,2,
4,2750,Lindo apartamento está localizado no bairro Sa...,R$ 1.250,2220,"Rua Garção Tinoco, Santana",90²,3,2,2,Destaque
...,...,...,...,...,...,...,...,...,...,...
95,4000,O apartamento está localizado no bairro Cabo B...,Null,Null,"Avenida Índio Arabutan, Cabo Branco",60²,2,1,1,Destaque
96,750,"Condomínio Portal das Américas, Golden Park, C...",R$ 350,Null,"Chacara Manella, Cambé",47²,2,Null,1,
97,650,O apartamento está localizado no bairro Farolâ...,R$ 80,565,"Avenida Josino José de Almeida, Farolândia",70²,3,Null,1,
98,2000,03 quartos com armários sendo 01 suíte com box...,R$ 200,Null,"Santa Mônica, Uberlândia",80²,3,2,2,
